# Setup Empty Pipeline with Screen


In [ ]:
# import resources
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume

# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length=60) > 50000

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen=universe)

# Setup Zipline Engine

In [ ]:
import helper
from zipline.utils.calendars import get_calendar
from zipline.data import bundles

# Name of bundle
EOD_BUNDLE_NAME = 'quantopian-quandl'

# Load the data bundle
bundle_data = bundles.load(EOD_BUNDLE_NAME)

# Setup the engine to look at the top 500 stocks who have had the highest rolling Average Dollar Volume
# over a 120-day window -- This is arbitrary and we can use this parameter to refine which stocks we
# want in our universe
universe = AverageDollarVolume(window_length=120).top(500) 

# Select the trading calendar that will be used as a reference when slicing the data
trading_calendar = get_calendar('NYSE') 

# Load the bundle we configured in the previous step into the engine
bundle_data = bundles.load(EOD_BUNDLE_NAME)

# Create the engine -- the details of this function are in the utils.py file
engine = helper.build_pipeline_engine(bundle_data, trading_calendar)

# View Pipeline as Graph

In [ ]:
import graphviz

# Render the pipeline as a DAG
pipeline.show_graph()

# Run Pipeline Engine

In [ ]:
import pandas as pd

# Set the start and end dates
start_date = pd.Timestamp('2016-01-05', tz = 'utc')
end_date = pd.Timestamp('2016-01-05', tz = 'utc')

# Run our pipeline for the given start and end dates
pipeline_output = engine.run_pipeline(pipeline, start_date, end_date)

# Get Universe Tickers

In [ ]:
# Get the values in index level 1 and save them to a list
universe_tickers = pipeline_output.index.get_level_values(1).values.tolist()

# Create Data Portal

In [ ]:
from zipline.data.data_portal import DataPortal

# Create a data portal
data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = bundle_data.equity_daily_bar_reader.first_trading_day,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)

# View Historical Data

Get the OHLC + V data for a given time period. This data will be split into individual dataframes.

In [ ]:
def get_field_data(data_portal, trading_calendar, assets, start_date, end_date, field):
    
    # Set the given start and end dates to Timestamps. The frequency string C is used to
    # indicate that a CustomBusinessDay DateOffset is used
    end_dt = pd.Timestamp(end_date, tz='UTC', freq='C')
    start_dt = pd.Timestamp(start_date, tz='UTC', freq='C')

    # Get the locations of the start and end dates
    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)

    # return the historical data for the given window
    return data_portal.get_history_window(assets=assets, end_dt=end_dt, bar_count=end_loc - start_loc,
                                          frequency='1d',
                                          field=field,
                                          data_frequency='daily')

# The window of data to obtain
start_date = '2015-01-05'
end_data = '2016-01-05'

# Get the open data
open_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                          start_date, end_date, 'open')

# Get the high data
high_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                              start_date, end_date, 'high')

# Get the low data
low_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                         start_date, end_date, 'low')

# Get the closing data
close_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                              start_date, end_date, 'close') 

# Get the volume data
volume_data = get_field_data(data_portal, trading_calendar, universe_tickers,
                            start_date, end_date, 'volume')

# Combine OHLC dataframes into singular dataframe
Here we combine the four individual dataframes representing OHLC + V data into one historical dataframe.

In [ ]:
import numpy as np

#Create dataframe and append blank row
historical_dfs = pd.DataFrame(columns=universe_tickers)
historical_dfs = historical_dfs.append(pd.Series([np.nan]), ignore_index=True)

# Loop through each universe ticker and create a combined dataframe for that ticker
for ticker in universe_tickers:
    # Get individual series representing the OHLCV data
    open_series = open_data[ticker]
    high_series = high_data[ticker]
    low_series = low_data[ticker]
    close_series = close_data[ticker]
    volume_series = volume_data[ticker]
    
    # Combine these series into 1 dataframe
    columns = ['open', 'high', 'low', 'close', 'volume']
    df = pd.concat([open_series,high_series, low_series, close_series, volume_series], axis=1)
    df.columns = columns    
    
    # Save this dataframe to historical_dfs
    historical_dfs[ticker] = pd.Series([df])
    
    
# Change the columns to be more human readable
columns = helper.beautify_tickers(universe_tickers)
historical_dfs.columns = columns

In [ ]:
from pypf.instrument import DataframeInstrument
from pypf.chart import PFChart

# Set up dataframe instrument
df = historical_dfs['ABM'][0]
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
df['Date'] = df.index.astype(str)
df['Date'] = df['Date'].str.slice(0,10)
df['Volume'] = df['Volume'].astype(int)
dfi = DataframeInstrument('ABM', dataframe=df)

# Create pf chart
chart = PFChart(dfi)
chart.create_chart()
print(chart.chart)

# Create Relative Strength Chart
Use the OHLC data for two stocks to create a RS chart. We will calculate the ratio between the closing price of the two securities on a daily basis, and record this price as all four values of OHLC in the new, combined dataframe. This dataframe can then be used to construct a P&F chart of these ratios, giving us a relative strength chart.

In [ ]:
# This function will generate an rs chart for two given securities
def generate_rs_chart(numerator, denominator, historical_dfs):
    ''' 
        This function will generate an RS chart of Numerator/Denominator usingn data from
        historical dfs
        
        @param numerator: (str) Numerator ticker symbol
        @param denominator: (str) Denominator ticker symbol
        @param historical_dfs: (pd.DataFrame) Dataframe where each col is a ticker, and first entry is OHLC data 
                                for that ticker.
    '''
    # Create joint dataframe to hold RS data
    rs_df = pd.DataFrame(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])
    
    # Set 'Close' equal to ratio between two close prices on a daily basis
    rs_df['Close'] = historical_dfs[numerator][0]['close'] / historical_dfs[denominator][0]['close']
    
    # Set other columns equal to same ratio (close/close), since this is the only value that matters
    rs_df['High'] = rs_df['Close']
    rs_df['Low'] = rs_df['Close']
    rs_df['Open'] = rs_df['Close']
    rs_df['Volume'] = rs_df['Close']
    
    # Format date for use with DataframeInstrument
    rs_df['Date'] = rs_df.index.astype(str)
    rs_df['Date'] = rs_df['Date'].str.slice(0,10)
    rs_df['Volume'] = rs_df['Volume'].astype(int)
    
    
    # Create a new DataframeInstrument using this new dataframe
    rs_instrument = DataframeInstrument('AAPL_MSFT', dataframe=rs_df)
    
    # Chart the RS data
    chart = PFChart(rs_instrument)
    chart.create_chart()
    print(chart.chart)

In [ ]:
# Generate a RS chart
generate_rs_chart('AAPL', 'MSFT', historical_dfs)

In [ ]:
'''
# Run if changes are made to instrument functions
import importlib

importlib.reload(pypf.instrument)
'''